In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from numba import njit

# Calculte Energy of site (i,j) Working Good.
@njit
def energy(lattice, i, j, size, J):

    #                               LEFT                         RIGHT                        UP                           DOWN
    energy = - J * lattice[i, j] * (lattice[i, (j - 1) % size] + lattice[i, (j + 1) % size] + lattice[(i - 1) % size, j] + lattice[(i + 1) % size, j])

    return energy

# Monte Carlo Step, Working Good.

@njit
def metropolis(lattice, beta, size, J):

    i = np.random.randint(0, size)
    j = np.random.randint(0, size)

    deltaE = -2 * energy(lattice, i, j, size, J)

    # Change the value of the spin as a function of the energy difference
    if deltaE < 0.0:
        lattice[i,j] *= -1
    elif np.random.random() < np.exp( - deltaE / beta):
        lattice[i,j] *= -1

# Calculate the magnetization of a given spin grid, Working Good

@njit
def magnetization(lattice, size):
    mag = np.sum(lattice) / size**2 # Taille ** 2 = Number of spins
    return np.abs(mag)

# Calculate the magnetic susceptibility for a given evolution of magnetization over time, Working Good.

@njit
def susceptibility(magnetizations, beta, size):
     return size * ( np.mean(magnetizations**2) - np.mean(magnetizations)**2 ) / beta

# Animate the evolution of spin over time, Working Good.


@njit
def cumulant(magnetizations):
     return 1 - ( np.mean(magnetizations**4) / (3 * (np.mean(magnetizations**2))**2) )
 
@njit
def runAnim(lattice, iterations, size, J, beta):
        
    allMag = np.zeros(iterations)
    allEnergy = np.zeros(iterations)
    allGrid = np.zeros((iterations, size, size))

    for i in range (iterations):
        mag = magnetization(lattice, size)
        Energy = 0 

        for j in range (0, size):
            for k in range (0, size):
                Energy += energy(lattice, j, k, size, J) / 2
        
        allMag[i] = mag
        allEnergy[i] = Energy
        allGrid[i] = lattice.copy()
        metropolis(lattice, beta, size, J)
         
    return allMag, allEnergy, allGrid

# Calculate the magnetization after a certain number of MTC Step, Working Good.
@njit
def runMag(lattice, iterations, size, beta, J):
    for i in range (iterations):
        metropolis(lattice, beta, size, J)
    return magnetization(lattice, size)

@njit
def runAll(size, Temp, J, kb):

    allT = np.empty(len(Temp))
    allMag = np.empty(len(Temp))
    allKhi = np.empty(len(Temp))

    for i in range(len(Temp)):
        allT[i] = Temp[i]
        beta = Temp[i] * kb
        print(i)
        np.random.seed(24032003)
        GrilleUp = np.ones((size, size))
        GrilleRandom = np.random.choice(np.array([-1, 1]), size=(size, size))

        tempMagUp = np.empty(0)
        tempMagRandom = np.empty(0)

        tempMagUp = np.append(tempMagUp, magnetization(GrilleUp, size))
        tempMagRandom = np.append(tempMagRandom, magnetization(GrilleRandom, size))
        iterations = 0
        while np.abs(np.mean(tempMagRandom[-10000:]) - np.mean(tempMagUp[-10000:])) > 0.00001:
            iterations += 1
            if iterations >= 1000000:
                print("Maximum iterations reached. Exiting loop.")
                
                break
            metropolis(GrilleUp, beta, size, J)
            metropolis(GrilleRandom, beta, size, J)
            tempMagUp = np.append(tempMagUp, magnetization(GrilleUp, size))
            tempMagRandom = np.append(tempMagRandom, magnetization(GrilleRandom, size))

        meanMag = (np.mean(tempMagRandom[-10000:]) + np.mean(tempMagUp[-10000:]) ) / 2
        allMag[i] = meanMag
        allKhi[i] = susceptibility( tempMagUp, beta, size)

    return allT, allMag, allKhi


In [2]:
kb = 1
Temp = np.array([1.40, 1.50, 1.60, 1.70, 1.80, 1.90, 2.0, 2.05, 2.10, 2.15, 2.20, 2.25, 2.30, 2.35, 2.40, 2.45, 2.50, 2.55, 2.60, 2.70, 2.80, 2.90, 3.00, 3.10, 3.20 ])
J = 1
beta = kb * Temp

size = 5
allT10, allMag10, allKhi10 = runAll(size, Temp, J, kb)

size = 10
allT20, allMag20, allKhi20 = runAll(size, Temp, J, kb)

size = 20
allT30, allMag30, allKhi30 = runAll(size, Temp, J, kb)

size = 30
allT40, allMag40, allKhi40 = runAll(size, Temp, J, kb)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5


In [ ]:
plt.figure()
plt.plot(allT10 / 2.27, allMag10, linestyle="None", marker=".", color = "black", label= "$N = 5$" )
plt.plot(allT20 / 2.27, allMag20, linestyle="None", marker="x", color = "blue", label= "$N = 10$")
plt.plot(allT30 / 2.27, allMag30, linestyle="None", marker="+", color = "green", label= "$N = 20$")
plt.plot(allT40 / 2.27, allMag40, linestyle="None", marker="*", color = "red", label= "$N = 30$")
plt.legend()
plt.xlabel("$T / T_C$")
plt.ylabel("$ M $")
plt.savefig("out/MagAll.pdf")


In [ ]:
plt.figure()
plt.plot(allT10 / 2.27, allKhi10, linestyle="None", marker="o", color = "black", label= "$N = 5$" )
plt.plot(allT20 / 2.27, allKhi20, linestyle="None", marker="^", color = "blue", label= "$N = 10$")
plt.plot(allT30 / 2.27, allKhi30, linestyle="None", marker="*", color = "green", label= "$N = 20$")
plt.plot(allT40 / 2.27, allKhi40, linestyle="None", marker="s", color = "red", label= "$N = 30$")
plt.legend()
plt.xlabel("$T / T_C")
plt.ylabel("$ \chi$")
plt.savefig("out/KhiAll.pdf")